Предварительный обзор данных из следующих источников (обновляется):

- реформаЖКХ
- данные исследования с хабра (основаны на данных реформаЖКХ) (1М записей)
- mos.ru

РеформаЖКХ: 

In [42]:
import pandas as pd
import time

In [ ]:
a = "id,region_id,area_id,city_id,street_id,shortname_region,formalname_region,shortname_area,formalname_area,shortname_city,formalname_city,shortname_street,formalname_street,house_number,building,block,letter,address,houseguid,built_year,exploitation_start_year,project_type,house_type,floor_count_max,floor_count_min,area_land"
dfcols = a.split(sep=',')

In [ ]:
df = pd.read_csv('export-reestrmkd-77-1.csv', usecols=dfcols)

In [ ]:
df.dtypes

In [ ]:
df.columns

In [ ]:
df['floor_count_max'].fillna(0, inplace=True)
df['floor_count_max'] = df['floor_count_max'].astype(int)


In [ ]:
df.houseguid = df.houseguid.str.upper()
df.houseguid.sort_values(ascending=False).head(5)

In [ ]:
len(df[df.floor_count_max == 0])

In [ ]:
df.floor_count_min.isna().sum()

In [ ]:
df[['id', 'floor_count_max', 'shortname_street', 'formalname_street', 'house_number', 'house_type', 'project_type']].sort_values('floor_count_max', ascending=False)

In [ ]:
height_groups = df.groupby('floor_count_max').size().sort_values(ascending=False)

In [ ]:
height_groups.plot(kind='bar')

In [ ]:
df.to_csv('export-reestrmkd-77-min.csv', encoding='utf-8')

1М подвыборка с хабры:

In [ ]:
df2 = pd.read_csv('./analyze-reformagkh/data.csv')

In [ ]:
df2.dtypes

In [ ]:
floor_heights = df2.groupby('floors').size()

In [ ]:
floor_heights.plot(kind='pie')

In [ ]:
floor_heights.plot(kind='bar')

Данные data.mos.ru:

In [ ]:
with open('./mos/structure.json', 'r', encoding='utf-8') as f:
    dft = pd.read_json('./mos/structure.json', encoding='utf-8')

In [ ]:
# кодировка исходника cp1251, код ниже переводит в utf-8
# причина - на чтении cp1251-файла падает pandas.DataFrame.read_json
for i in range(1,7):
    with open('./mos/data-29580-2019-01-20-'+str(i)+'.json', 'r', encoding='cp1251') as fc:
        with open('./mos/data-29580-2019-01-20-'+str(i)+'-utf8.json', 'w+', encoding='utf-8') as fu:
            fu.write(fc.read())
            
# TODO: переписать пути через os.

In [ ]:
dcols3 = 'UNOM,ADDRESS,ADM_AREA,DISTRICT,NREG,N_FIAS,geoData'.split(sep=',')

In [ ]:
df3 = pd.DataFrame()
# %timeit не работает, делаем по-дедовски
t0 = time.time()
for i in range(1,2):
    with open('./mos/data-29580-2019-01-20-'+str(i)+'-utf8.json', 'r', encoding='utf-8') as f:
        dft = pd.read_json(f, encoding='utf-8')
        df3 = pd.concat([df3, dft])
        print(len(df3))
        # del(dft)
t1 = time.time()
print(t1-t0)

In [ ]:
for i in range(0,len(dft['Columns'])):
    print(dft['Columns'][i])

In [ ]:
df3.N_FIAS = df3.N_FIAS.str.upper()

In [ ]:
df3.N_FIAS.sort_values(ascending=False).head(5)

In [ ]:
df.houseguid.sort_values(ascending=False).head(5)

In [ ]:
df3.dtypes

In [ ]:
df3.head(3)

Число записей с непустыми геоданными:

In [ ]:
df3.geoData.agg(pd.notna).sum()

Без геоданных:

In [ ]:
df3.geoData.isna().sum()

In [ ]:
d = df3.merge(df, how='inner', right_on='houseguid', left_on='N_FIAS')

In [ ]:
d.columns

In [ ]:
res0 = d[['floor_count_max', 'geoData', 'floor_count_min', 'ADDRESS', 'built_year', 'exploitation_start_year', 'project_type', 'house_type', 'area_land']]

In [ ]:
res0.to_csv('res0.csv', encoding='utf-8')

OSM XML в области, окаймляющей МКАД

In [5]:
import xml.etree.ElementTree as ET

In [6]:
tree = ET.parse('./map')
root = tree.getroot()

Здания в XML существуют в виде `nodes` - точек на карте и в виде `relations` - геометрических отношений, обозначающих их контур. Простых "точек" в сете не более 1.5 тысяч:

In [44]:
# be effective is my hobby
b = 0
l = 0
h = 0
n = 0
t0 = time.time()
for node in root.iter('node'):
    for tag in node.iter('tag'):
        if 'building' in tag.attrib['k']:
            b = b + 1
            if 'levels' in tag.attrib['k']:
                l = l + 1
            if 'height' in tag.attrib['k']:
                h = h + 1
    n = n + 1
t1 = time.time()
print('timed', str(t1-t0), 'sec')
print('all', str(n))
print('buildings', str(b))
print('with levels', str(l))
print('with heights', str(h))

timed 41.33345079421997 sec
all 3206997
buildings 1146
with levels 349
with heights 0


In [77]:
t = True
t0=time.time()
table = []

for r in root.iter('relation'):
    
    for tag in r.iter('tag'):
        if tag.attrib['k'] == 'building':
            table.append(r)
            pass
    
t1=time.time()

In [51]:
#tag "building" count
b = 156111
#tag "building:levels" count
l = 84877

84877


In [ ]:
table = []
line = dict{'lat', 'lon', 'levels', 'height', 'street', ''}
for tag in root.iter('tag'):
    if tag.attrib['k'] == 'building':
        